# Improving Brewster Paper

In the original paper **General characterization of qubit-preserving impairments on two-qubit Bell nonlocality**, a technique was presented that would correct errors such as polarization misalignments of entangled qubits by optimizing a set of unitary gets for each of the qubits Alice and Bob have. A follow up paper, **Automated Bell inequality violation searches for estimating entanglement quality in fiber**, determined that the Nelder-Mead Method was deemed to be the best method for optimization, though Bayesian Optimization was also quoted as being a close second. 

Previous work on this repo has simulated CHSH games on a quantum network using SeQuEnCe, with more recent additions implementing polarization misalignment and the proposed corrective technique by the above papers. However, we believe that we can improve this process further by utilizing more suitable optimization techniques. The below experiment uses COnstrained Optimization BY Linear Approximation, or COBYLA, to better approximate better alphas in a noisy quantum environment.  

In [1]:
import network_module
from scipy.optimize import minimize
import numpy as np

In [2]:
theta1 = 0.7854          
theta2 = 0          
theta3 = -0.3927        
theta4 = 0.3927

In [3]:
#Bayesian Optimization function
from bayes_opt import BayesianOptimization
import numpy as np

def BayesOPT(deltas,fidelity=1000):
    iteration_s_values = []

    #play chsh game
    def simulated_chsh(alpha1, alpha2, alpha3, alpha4, alpha5, alpha6):
        g = network_module.Game(deltas = deltas,fid = 0.9)
        alphas_alice = [alpha1,alpha2,alpha3]
        alphas_bob = [alpha4,alpha5,alpha6]
        result = g.start(fidelity,theta1,theta2,theta3,theta4,alphas_alice,alphas_bob)
        S = g.referee.compute_chsh_s()
        iteration_s_values.append(S)
        return S

    # Define Bayesian optimizer bounds for Alice’s PC angles (voltages mapped to angles)
    pbounds = {
        'alpha1': (-np.pi, np.pi),
        'alpha2': (-np.pi, np.pi),
        'alpha3': (-np.pi, np.pi),
        'alpha4': (-np.pi, np.pi),
        'alpha5': (-np.pi, np.pi),
        'alpha6': (-np.pi, np.pi),
    }

    # Create optimizer
    optimizer = BayesianOptimization(
        f=simulated_chsh,
        pbounds=pbounds,
        verbose=0
    )

    import time

    start_time = time.time()
    # Run optimization manually with acquisition function
    optimizer.maximize(
        init_points=4,
        n_iter=26
    )


    end_time = time.time()
    return list(optimizer.max['params'].values()),end_time - start_time,np.maximum.accumulate(iteration_s_values)

In [4]:
from scipy.optimize import minimize
import numpy as np
import time

def NelderMeadOPT(deltas, fidelity=1000):
    iteration_s_values = []

    # Objective function (negative S, since Nelder-Mead minimizes)
    def simulated_chsh(angles):
        alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = angles
        g = network_module.Game(deltas=deltas, fid=0.9)
        alphas_alice = [alpha1, alpha2, alpha3]
        alphas_bob = [alpha4, alpha5, alpha6]
        result = g.start(fidelity, theta1, theta2, theta3, theta4, alphas_alice, alphas_bob)
        S = g.referee.compute_chsh_s()
        iteration_s_values.append(S)
        # print("S:",S)
        return -S  # Negative because we want to maximize S

    # Initial guess (can be random or zeros)
    x0 = np.random.uniform(-np.pi, np.pi, size=6)

    # Run optimization
    start_time = time.time()
    result = minimize(
        simulated_chsh,
        x0,
        method='Nelder-Mead',
        options={'maxiter': 30, 'disp': True}
    )
    end_time = time.time()
    
    best_angles = result.x.tolist()
    best_time = end_time - start_time
    s_curve = np.maximum.accumulate(iteration_s_values)
    

    return best_angles, best_time, s_curve


In [5]:
# deltaA = np.random.uniform(-np.pi, np.pi, size=3).tolist()
# deltaB = np.random.uniform(-np.pi, np.pi, size=3).tolist()
# deltas = [deltaA ,deltaB]

In [6]:
#COBYLA function
import numpy as np
from scipy.optimize import minimize

# Number of shots per evaluation
# shots = 2500
COBYLA_evals = []
COBYLA_progress = []
def COBYLA_OPT(deltas,fidelity=1000):
    def objective(x):
        alphas_alice = x[0:3]
        alphas_bob   = x[3:6]

        # Start a new game for each evaluation to avoid stale state
        g = network_module.Game(deltas = deltas,guarantee_entanglement = True,fid = 0.9)
        result = g.start(fidelity, theta1, theta2, theta3, theta4, alphas_alice, alphas_bob)
        S = g.referee.compute_chsh_s()
        print(result,S)
        COBYLA_evals.append(S)
        best_so_far = max(COBYLA_evals)
        COBYLA_progress.append(best_so_far)
    
        return -S  # minimize negative S to maximize S

# Construct COBYLA inequality constraints for [-pi, pi]
    pi_val = np.pi
    cons = []
    for i in range(6):
    # x[i] >= -pi  ->  x[i] + pi >= 0
        cons.append({'type': 'ineq', 'fun': lambda x, i=i: x[i] + pi_val})
    # x[i] <= pi   ->  pi - x[i] >= 0
        cons.append({'type': 'ineq', 'fun': lambda x, i=i: pi_val - x[i]})

    x0 = np.random.uniform(-np.pi, np.pi, size=6)
    res = minimize(objective, x0, method='COBYLA',constraints=cons,
               options={'rhobeg': 1.0, 'tol': 1e-6, 'maxiter': 30})

    return x0

In [7]:
#Experiments
import time
num_exp = 5

#convergence progress
BO_conv = []
NM_conv = []
COB_conv = []

#validation results
BO_res = []
NM_res = []
COB_res = []

#Time results
BO_time = []
NM_time = []
COB_time = []

for x in range(num_exp):
    print("Iteration:",x)
    deltaA = np.random.uniform(-np.pi, np.pi, size=3).tolist()
    deltaB = np.random.uniform(-np.pi, np.pi, size=3).tolist()
    deltas = [deltaA ,deltaB]

    #Bayesian Optimization
    # start_time = time.time()
    # X_high_fid, time_high_fid,progress = BayesOPT(deltas,fidelity=2500)
    # end_time = time.time()
    # BO_conv.append(progress)
    # BO_time.append(end_time-start_time)
    # alphas_alice = X_high_fid[0:3]
    # alphas_bob = X_high_fid[3:6]

    # #Verifying "True" value of Baysian Optimization
    # g = network_module.Game(deltas = deltas,guarantee_entanglement = True,fid = 0.9)
    # g.start(10000,theta1,theta2,theta3,theta4,alphas_alice,alphas_bob)
    # S_high = g.referee.compute_chsh_s()
    # # print("S HIGH FID:",S_high)
    # BO_res.append(S_high)
    
    # start_time = time.time()
    # X_high_fid, time_high_fid,progress = NelderMeadOPT(deltas,fidelity=2500)
    # end_time = time.time()
    # NM_conv.append(progress)
    # NM_time.append(end_time-start_time)
    # alphas_alice = X_high_fid[0:3]
    # alphas_bob = X_high_fid[3:6]

    # #Verifying "True" value of NelderMeadOPT
    # g = network_module.Game(deltas = deltas,guarantee_entanglement = True,fid = 0.9)
    # g.start(10000,theta1,theta2,theta3,theta4,alphas_alice,alphas_bob)
    # S_high = g.referee.compute_chsh_s()
    # print("S HIGH FID:",S_high)
    # NM_res.append(S_high)

    #COBYLA
    COBYLA_evals = []
    COBYLA_progress = []
    start_time = time.time()
    # x0 = np.zeros(6)
    # res = minimize(objective, x0, method='COBYLA',constraints=cons,
    #            options={'rhobeg': 1.0, 'tol': 1e-6, 'maxiter': 30})
    # print(deltas)
    res = COBYLA_OPT(deltas,fidelity=2500)
    # print(deltas)
    end_time = time.time()
    COB_time.append(end_time-start_time)
    COB_conv.append(COBYLA_progress)

    #Verifying "True" value of COBYLA
    alphas_alice = res[0:3]
    alphas_bob = res[3:]
    # print(deltas)
    g = network_module.Game(deltas = deltas,guarantee_entanglement = True,fid = 0.9)
    
    result = g.start(10000, theta1, theta2, theta3, theta4, alphas_alice, alphas_bob)
    S = g.referee.compute_chsh_s()
    print("Win rate:",result,"S:",S)
    # if result < 0.7:
    #     g = network_module.Game(deltas = deltas,guarantee_entanglement = True,fid = 0.9,debug=True)
    #     result = g.start(100, theta1, theta2, theta3, theta4, alphas_alice, alphas_bob)
    #     break
    
    print(S)
    COB_res.append(S)

    

Iteration: 0

[CHSH Stats]
  Setting (0, 0): E = -0.815, winrate = 0.092, rounds = 628
  Setting (0, 1): E = 0.065, winrate = 0.533, rounds = 599
  Setting (1, 0): E = -0.067, winrate = 0.466, rounds = 641
  Setting (1, 1): E = 0.424, winrate = 0.288, rounds = 632
Global S = -1.2413, Global winrate = 0.3432

0.3432 -1.2413114262322291

[CHSH Stats]
  Setting (0, 0): E = 0.164, winrate = 0.582, rounds = 622
  Setting (0, 1): E = -0.800, winrate = 0.100, rounds = 650
  Setting (1, 0): E = 0.063, winrate = 0.531, rounds = 638
  Setting (1, 1): E = 0.302, winrate = 0.349, rounds = 590
Global S = -0.8750, Global winrate = 0.3888

0.3888 -0.8750118522256815

[CHSH Stats]
  Setting (0, 0): E = 0.006, winrate = 0.503, rounds = 654
  Setting (0, 1): E = 0.552, winrate = 0.776, rounds = 612
  Setting (1, 0): E = -0.239, winrate = 0.380, rounds = 610
  Setting (1, 1): E = 0.609, winrate = 0.196, rounds = 624
Global S = -0.2899, Global winrate = 0.4632

0.4632 -0.2899148316190241

[CHSH Stats]
  S

KeyboardInterrupt: 

In [ ]:
import statistics
print("Mean validation Bayesian Optimization:",statistics.mean(BO_res))
print("Standard Deviation Bayesian Optimization:",statistics.stdev(BO_res))

In [ ]:
print("Mean validation Nelder-Mead:",statistics.mean(NM_res))
print("Standard Deviation Nelder-Mead:",statistics.stdev(NM_res))

In [ ]:
print("Mean validation COBYLA:",statistics.mean(COB_res))
print("Standard Deviation COBYLA:",statistics.stdev(COB_res))

In [ ]:
BO_mean = np.array(BO_conv).mean(axis=0)
BO_std  = np.array(BO_conv).std(axis=0)

# NM_mean = NM_conv.mean(axis=0)
# NM_std  = np.array(NM_conv).std(axis=0)

COB_mean = np.array(COB_conv).mean(axis=0)
COB_std  = np.array(COB_conv).std(axis=0)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))

# Bayesian Optimization
plt.plot(BO_mean, label='BO mean')
plt.fill_between(range(len(BO_mean)), BO_mean-BO_std, BO_mean+BO_std, alpha=0.3)

# #Nelder-Mead
# plt.plot(NM_mean, label='COBYLA mean')
# plt.fill_between(range(len(NM_mean)), NM_mean-NM_std , NM_mean+NM_std, alpha=0.3)

# COBYLA
plt.plot(COB_mean, label='COBYLA mean')
plt.fill_between(range(len(COB_mean)), COB_mean-COB_std, COB_mean+COB_std, alpha=0.3)
plt.xlim(0, 29)
plt.xlabel('Iteration')
plt.ylabel('Best S so far')
plt.title('Convergence (Mean ± Std) over Multiple Experiments')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
BO_time

In [ ]:
COB_time